# Segment, localise and track


In [1]:
from macrohet import dataio, tile
import numpy as np
from tqdm.auto import tqdm
from cellpose import models
import btrack 
import torch
import dask.array as da
import os
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
# defining cellpose model to use
model = models.Cellpose(
                        gpu=True, 
                        model_type='cyto', 
                        net_avg=True, 
                        device=torch.device('cuda')
                        )

Using device: cuda

NVIDIA RTX A6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


### Define functions to tidy up main block of code

In [3]:
def segment(frame, model = model, channels = [0,0], diameter = 325, min_size = 2500):
    
    masks, flows, styles, diams = model.eval(frame, 
                                             channels = channels, 
                                             diameter = diameter, 
                                             min_size = min_size, 
                                             )
    return masks

# define features to use for tracking 
features = [
  "area",
  "major_axis_length",
  "minor_axis_length",
  "orientation",
  "mean_intensity",
    ]

def localise(masks, intensity_image, properties=tuple(features), use_weighted_centroid = False):
    
    # localise objs in images
    objects = btrack.utils.segmentation_to_objects(masks,
                                                   intensity_image, 
                                                   properties=properties,
                                                   use_weighted_centroid=use_weighted_centroid, 
                                                   )
    return objects

# define config fn to use
config_fn = '/home/dayn/analysis/BayesianTracker/models/upscale_cell_config.json'

def track(objects, masks, config_fn, search_radius = 20):

    # initialise a tracker session using a context manager
    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure(config_fn)
        # set max search radius
        tracker.max_search_radius = search_radius
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # redefine features so that both channels are included in track measurements
        tracker.features = list(objects[0].properties.keys())
        # append the objects to be tracked
        tracker.append(objects)
        # set the tracking volume
        tracker.volume=((0, masks.shape[-2]), (0, masks.shape[-1]))
        # track them (in interactive mode)
        tracker.track(step_size=25)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # store the tracks
        tracks = tracker.tracks

    return tracks

### Load experiment of choice

The Opera Phenix is a high-throughput confocal microscope that acquires very large 5-dimensional (TCZXY) images over several fields of view in any one experiment. Therefore, a lazy-loading approach is chosen to mosaic, view and annotate these images. This approach depends upon Dask and DaskFusion. The first step is to load the main metadata file (typically called `Index.idx.xml` and located in the main `Images` directory) that contains the image filenames and associated TCXZY information used to organise the images.

In [4]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet_nemo/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


### View assay layout and mask information (optional)

The Opera Phenix acquires many time lapse series from a range of positions. The first step is to inspect the image metadata, presented in the form of an `Assaylayout/experiment_ID.xml` file, to show which positions correspond to which experimental assays.

In [5]:
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)# mask_exist=True,  image_dir = image_dir, image_metadata = metadata)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

# Implement upscaled localisation

In [6]:
# select one test expt to work with 
row, column = 3, 5

In [38]:
# testing on subset of whole dataset
n_frames_test = 2

In [97]:
tracks_fn = os.path.join(base_dir, f'labels/full_localisation/{row,column}_objs_masks_tracks.h5')
with btrack.io.HDF5FileHandler(tracks_fn, 'r') as hdf:
    masks = hdf.segmentation

[INFO][2023/04/27 03:00:35 pm] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5...
27-Apr-23 15:00:35 - btrack.io.hdf - INFO     - Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5...
[INFO][2023/04/27 03:00:45 pm] Loading segmentation (75, 6048, 6048)
27-Apr-23 15:00:45 - btrack.io.hdf - INFO     - Loading segmentation (75, 6048, 6048)
[INFO][2023/04/27 03:00:45 pm] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5
27-Apr-23 15:00:45 - btrack.io.hdf - INFO     - Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/(3, 5)_objs_masks_tracks.h5


In [61]:
image_dir = os.path.join(base_dir, 'macrohet_images/Images')
images = tile.compile_mosaic(image_dir, 
                             metadata, 
                             row, 
                             column, 
                             set_plane = 'sum_proj',
                             ).astype(np.uint16)

# reshape intensity image to be gfp, rfp on last axis for regionprops
intensity_image = np.stack([images[:,0,...], 
                            images[:,1,...]], axis = -1)

In [62]:
%%time
# localise objs in images
objects = btrack.utils.segmentation_to_objects(segmentation = masks,
                                               intensity_image = intensity_image, 
                                               properties=tuple(features),
                                               scale=(1/(5.04),1/(5.04)),
                                               use_weighted_centroid=False, 
                                               )

[INFO][2023/04/27 02:19:16 PM] Localizing objects from segmentation...
100%|█████████████████████████████████████████████████████████████████████| 75/75 [15:10<00:00, 12.14s/it]
[INFO][2023/04/27 02:34:27 PM] Objects are of type: <class 'dict'>
[INFO][2023/04/27 02:34:27 PM] ...Found 42674 objects in 75 frames.


CPU times: user 3h 36min 35s, sys: 16min 56s, total: 3h 53min 32s
Wall time: 15min 11s


In [103]:
# define config fn to use
config_fn = '/home/dayn/analysis/BayesianTracker/models/cell_config.json' 
#'/home/dayn/analysis/btrack/models/cell_config_pnassign.json'
#'/home/dayn/analysis/BayesianTracker/models/upscale_cell_config.json'
search_radius = 20

# initialise a tracker session using a context manager
with btrack.BayesianTracker() as tracker:
    # configure the tracker using a config file
    tracker.configure(config_fn)
    # set max search radius
    tracker.max_search_radius = search_radius
    # define tracking method
    tracker.tracking_updates = ["MOTION", "VISUAL"]
    # redefine features so that both channels are included in track measurements
    tracker.features = list(objects[0].properties.keys())
    # append the objects to be tracked
    tracker.append(objects)
    # set the tracking volume
    tracker.volume=((0, masks.shape[-2]), (0, masks.shape[-1]))
    # track them (in interactive mode)
    tracker.track(step_size=25)
    # generate hypotheses and run the global optimizer
    tracker.optimize()
    # store the tracks
    tracks_cell_config = tracker.tracks

previous_config_tracks, properties, graph = btrack.utils.tracks_to_napari(tracks_cell_config, ndim=2)


[INFO][2023/04/27 05:16:22 pm] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
27-Apr-23 17:16:22 - btrack.libwrapper - INFO     - Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/04/27 05:16:22 pm] Starting BayesianTracker session
27-Apr-23 17:16:22 - btrack.core - INFO     - Starting BayesianTracker session
[INFO][2023/04/27 05:16:22 pm] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/cell_config.json
27-Apr-23 17:16:22 - btrack.config - INFO     - Loading configuration file: /home/dayn/analysis/BayesianTracker/models/cell_config.json
[INFO][2023/04/27 05:16:22 pm] Objects are of type: <class 'list'>
27-Apr-23 17:16:22 - btrack.io.utils - INFO     - Objects are of type: <class 'list'>
[INFO][2023/04/27 05:16:23 pm] Starting tracking... 
27-Apr-23 17:16:23 - btrack.core - INFO     - Starting tracking... 
[INFO][2023/04/27 05:16:23 pm] Update using: ['VISUAL', 'MOTION']
27-Apr-23 17:16:23 - btrack.core - INFO    

GLPK Integer Optimizer 5.0
39356 rows, 2033174 columns, 6026821 non-zeros
2033174 integer variables, all of which are binary
Preprocessing...
19678 rows, 2033174 columns, 6026821 non-zeros
2033174 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 19678
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
19678 rows, 2033174 columns, 6026821 non-zeros
*     0: obj =   1.204776960e+05 inf =   0.000e+00 (2011658)
*    95: obj =   1.194003960e+05 inf =   0.000e+00 (1877357)
*   246: obj =   1.182632460e+05 inf =   0.000e+00 (1076132)
Perturbing LP to avoid stalling [332]...
*   362: obj =   1.176846960e+05 inf =   0.000e+00 (629792)
*   487: obj =   1.167869460e+05 inf =   0.000e+00 (480243)
*   708: obj =   1.147121460e+05 inf =   0.000e+00 (457788)
*   839: obj =   1.137545460e+05 inf =   0.000e+00 (452485)
*  1072: obj =   1

[INFO][2023/04/27 06:19:41 pm] Ending BayesianTracker session
27-Apr-23 18:19:41 - btrack.core - INFO     - Ending BayesianTracker session


### Load downscale tracks for comparison

In [64]:
ds_tracks_fn = os.path.join(base_dir, f'labels/full_localisation/downscaled/{row,column}_downsc_objs_masks_tracks.h5')
with btrack.io.HDF5FileHandler(ds_tracks_fn, 'r') as hdf:
    ds_tracks = hdf.tracks
    ds_segmentation = hdf.segmentation
ds_napari_tracks, properties, graph = btrack.utils.tracks_to_napari(ds_tracks, ndim=2)

[INFO][2023/04/27 02:35:42 PM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(3, 5)_downsc_objs_masks_tracks.h5...
[INFO][2023/04/27 02:35:42 PM] Loading tracks/obj_type_1
[INFO][2023/04/27 02:35:42 PM] Loading LBEP/obj_type_1
[INFO][2023/04/27 02:35:42 PM] Loading objects/obj_type_1 (26917, 5) (26917 filtered: None)
[INFO][2023/04/27 02:35:42 PM] Loading segmentation (75, 1200, 1200)
[INFO][2023/04/27 02:35:42 PM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(3, 5)_downsc_objs_masks_tracks.h5


In [66]:
import napari

In [67]:
viewer = napari.Viewer()

/home/dayn/analysis/btrack/btrack/dataio.py:3: UserWarning: `btrack.dataio` has been deprecated. Please use `btrack.io` subpackage instead.
  warnings.warn(  # noqa: B028


Assistant skips harvesting pyclesperanto as it's not installed.


/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari_tools_menu/__init__.py:194: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


In [68]:
viewer.add_image(images, 
                 channel_axis=1,
                 name=["macrophage", "mtb"],
                 colormap=["green",  "magenta"],
                 contrast_limits=[[100, 6000], [100, 2000]], 
                 visible = False
                )

[<Image layer 'macrophage' at 0x7f2b121ba670>,
 <Image layer 'mtb' at 0x7f2b0039f940>]

In [94]:
ds_masks_up_prop = btrack.utils.update_segmentation(ds_segmentation, tracks,)
ds_masks_ds_prop = btrack.utils.update_segmentation(ds_segmentation, ds_tracks,)

In [95]:
viewer.add_labels(masks, name = 'upscale masks')
viewer.add_labels(ds_masks_up_prop, name = 'ds_masks_up_prop')
viewer.add_tracks(napari_tracks, name = 'ds tracks up prop ')

<Tracks layer 'ds tracks up prop ' at 0x7f2b0522df70>

In [102]:
viewer.add_tracks(napari_tracks_cell_config, name = 'cell config')

<Tracks layer 'cell config' at 0x7f2b01feceb0>

In [96]:
viewer.add_labels(ds_segmentation, name = 'downscale masks')
viewer.add_tracks(ds_napari_tracks, name = 'downscale tracks')

<Tracks layer 'downscale tracks' at 0x7f2b06416400>

In [74]:
objects[0].properties['mean_intensity']

array([1194.51385127,  344.16593015])

In [75]:
images.dtype

dtype('uint16')

# Segment, localise and track

In [147]:
### iterate over positions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):

    # tile images
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
                                 set_plane = 'sum_proj',
                                 ).astype(np.uint16)

    # segment images from gfp channel only, dask to match intensity image
    masks = da.stack([segment(frame) 
                      for frame in tqdm(images[:,0,...], 
                                        desc = 'Segmenting')])

    # reshape intensity image to be gfp, rfp on last axis for regionprops
    intensity_image = np.stack([images[:,0,...], 
                                images[:,1,...]], axis = -1)

    # localise objects
    objects = localise(masks, 
                       intensity_image, 
                       )

    # filter out small objects
    objects = [o for o in objects if o.properties['area'] > 500]

    # track on upscaled config fn
    tracks = track(objects, masks, config_fn, search_radius = 100)

    # save out 
    with btrack.dataio.HDF5FileHandler(f'/mnt/DATA/macrohet/labels/full_localisation/upscale_{row, column}.h5', 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
            writer.write_objects(objects)
            writer.write_tracks(tracks)
            writer.write_segmentation(masks)


Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

Segmenting:   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/04/11 07:50:59 PM] Localizing objects from segmentation...
[INFO][2023/04/11 07:50:59 PM] Found intensity_image data
[INFO][2023/04/11 08:05:49 PM] Objects are of type: <class 'dict'>
[INFO][2023/04/11 08:05:49 PM] ...Found 43409 objects in 75 frames.
[INFO][2023/04/11 08:05:49 PM] Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2023/04/11 08:05:49 PM] btrack (v0.5.0) library imported
[INFO][2023/04/11 08:05:49 PM] Starting BayesianTracker session
[INFO][2023/04/11 08:05:49 PM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/cell_config.json
[INFO][2023/04/11 08:05:49 PM] Objects are of type: <class 'list'>
[INFO][2023/04/11 08:05:50 PM] Starting tracking... 
[INFO][2023/04/11 08:05:50 PM] Update using: ['VISUAL', 'MOTION']
[INFO][2023/04/11 08:05:51 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2023/04/11 08:05:58 PM]  - Timing (Bayesian updates: 69.15ms, Linking: 2.83ms)
[INFO][2023/04/11 08:05:58 PM]  

GLPK Integer Optimizer 5.0
23724 rows, 18809 columns, 25769 non-zeros
18809 integer variables, all of which are binary
Preprocessing...
11862 rows, 18809 columns, 25769 non-zeros
18809 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 11862
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
11862 rows, 18809 columns, 25769 non-zeros
*     0: obj =   6.628285404e+04 inf =   0.000e+00 (3713)
Perturbing LP to avoid stalling [1651]...
Removing LP perturbation [3715]...
*  3715: obj =   4.394459104e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  3715: mip =     not found yet >=              -inf        (1; 0)
+  3715: >>>>>   4.394459104e+04 >=   4.394459104e+04   0.0% (1; 0)
+  3715: mip =   4.394459104e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPT

[INFO][2023/04/11 08:06:08 PM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/upscale_(3, 4).h5...
[INFO][2023/04/11 08:06:08 PM] Writing objects/obj_type_1
[INFO][2023/04/11 08:06:08 PM] Writing labels/obj_type_1
[INFO][2023/04/11 08:06:08 PM] Loading objects/obj_type_1 (29635, 5) (29635 filtered: None)
[INFO][2023/04/11 08:06:08 PM] Writing properties/obj_type_1/area (29635,)
[INFO][2023/04/11 08:06:08 PM] Writing properties/obj_type_1/major_axis_length (29635,)
[INFO][2023/04/11 08:06:08 PM] Writing properties/obj_type_1/minor_axis_length (29635,)
[INFO][2023/04/11 08:06:08 PM] Writing properties/obj_type_1/orientation (29635,)
[INFO][2023/04/11 08:06:08 PM] Writing properties/obj_type_1/mean_intensity-0 (29635,)
[INFO][2023/04/11 08:06:08 PM] Writing properties/obj_type_1/mean_intensity-1 (29635,)
[INFO][2023/04/11 08:06:08 PM] Writing tracks/obj_type_1
[INFO][2023/04/11 08:06:08 PM] Writing dummies/obj_type_1
[INFO][2023/04/11 08:06:08 PM] Writing LBEP/obj_type_1
[I

...

[INFO][2023/04/13 12:12:11 PM] Ending BayesianTracker session
[INFO][2023/04/13 12:12:11 PM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/upscale_(5, 8).h5...
[INFO][2023/04/13 12:12:12 PM] Writing objects/obj_type_1
[INFO][2023/04/13 12:12:12 PM] Writing labels/obj_type_1
[INFO][2023/04/13 12:12:12 PM] Loading objects/obj_type_1 (31522, 5) (31522 filtered: None)
[INFO][2023/04/13 12:12:13 PM] Writing properties/obj_type_1/area (31522,)
[INFO][2023/04/13 12:12:13 PM] Writing properties/obj_type_1/major_axis_length (31522,)
[INFO][2023/04/13 12:12:13 PM] Writing properties/obj_type_1/minor_axis_length (31522,)
[INFO][2023/04/13 12:12:13 PM] Writing properties/obj_type_1/orientation (31522,)
[INFO][2023/04/13 12:12:13 PM] Writing properties/obj_type_1/mean_intensity-0 (31522,)
[INFO][2023/04/13 12:12:13 PM] Writing properties/obj_type_1/mean_intensity-1 (31522,)
[INFO][2023/04/13 12:12:13 PM] Writing tracks/obj_type_1
[INFO][2023/04/13 12:12:13 PM] Writing dummies/obj_ty

Segmenting:   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/04/13 02:33:43 PM] Localizing objects from segmentation...
[INFO][2023/04/13 02:33:43 PM] Found intensity_image data
[INFO][2023/04/13 02:48:11 PM] Objects are of type: <class 'dict'>
[INFO][2023/04/13 02:48:12 PM] ...Found 36860 objects in 75 frames.
[INFO][2023/04/13 02:48:12 PM] Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3397, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_529848/217926563.py", line 32, in <cell line: 2>
    tracks = track(objects, masks, config_fn, search_radius = 100)
  File "/tmp/ipykernel_529848/179702834.py", line 36, in track
    with btrack.BayesianTracker() as tracker:
  File "/home/dayn/analysis/BayesianTracker/btrack/core.py", line 138, in __init__
  File "/home/dayn/analysis/BayesianTracker/btrack/constants.py", line 14, in get_version_tuple
  File "/home/dayn/analysis/BayesianTracker/btrack/constants.py", line 8, in get_version
    MAX_LOST = 5
FileNotFoundError: [Errno 2] No such file or directory: '/home/dayn/analysis/BayesianTracker/btrack/VERSION.txt'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs

In [168]:
!pip install btrack==0.4.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.9 MB/s eta 0:00:00m eta 0:00:0100:01
  Attempting uninstall: btrack
    Found existing installation: btrack 0.5.2
    Uninstalling btrack-0.5.2:
      Successfully uninstalled btrack-0.5.2


In [174]:
import importlib
importlib.reload(btrack)

<module 'btrack' from '/home/dayn/analysis/BayesianTracker/btrack/__init__.py'>

In [175]:
btrack.__version__

'0.5.0'

In [176]:
# track on upscaled config fn
tracks = track(objects, masks, config_fn, search_radius = 100)

# save out 
with btrack.dataio.HDF5FileHandler(f'/mnt/DATA/macrohet/labels/full_localisation/upscale_{row, column}.h5', 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:
        writer.write_objects(objects)
        writer.write_tracks(tracks)
        writer.write_segmentation(masks)

[INFO][2023/04/14 09:40:29 AM] Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2023/04/14 09:40:29 AM] btrack (v0.5.0) library imported
[INFO][2023/04/14 09:40:29 AM] Starting BayesianTracker session
[INFO][2023/04/14 09:40:29 AM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/cell_config.json
[INFO][2023/04/14 09:40:29 AM] Objects are of type: <class 'list'>
[INFO][2023/04/14 09:40:30 AM] Starting tracking... 
[INFO][2023/04/14 09:40:30 AM] Update using: ['VISUAL', 'MOTION']
[INFO][2023/04/14 09:40:30 AM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2023/04/14 09:40:36 AM]  - Timing (Bayesian updates: 56.46ms, Linking: 3.05ms)
[INFO][2023/04/14 09:40:36 AM]  - Probabilities (Link: 0.99942, Lost: 1.00000)
[INFO][2023/04/14 09:40:36 AM]  - Stats (Active: 812, Lost: 9408, Conflicts resolved: 4387)
[INFO][2023/04/14 09:40:36 AM] Tracking objects in frames 25 to 49 (of 75)...
[INFO][2023/04/14 09:40:40 AM]  - Timing (Bayes

GLPK Integer Optimizer 5.0
19844 rows, 15742 columns, 21576 non-zeros
15742 integer variables, all of which are binary
Preprocessing...
9922 rows, 15742 columns, 21576 non-zeros
15742 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 9922
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
9922 rows, 15742 columns, 21576 non-zeros
*     0: obj =   5.629823487e+04 inf =   0.000e+00 (3250)
Perturbing LP to avoid stalling [1550]...
Removing LP perturbation [3276]...
*  3276: obj =   3.546716761e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  3276: mip =     not found yet >=              -inf        (1; 0)
+  3276: >>>>>   3.546716761e+04 >=   3.546716761e+04   0.0% (1; 0)
+  3276: mip =   3.546716761e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMA

[INFO][2023/04/14 09:40:43 AM] Ending BayesianTracker session
[INFO][2023/04/14 09:40:44 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/upscale_(5, 9).h5...
[INFO][2023/04/14 09:40:44 AM] Writing objects/obj_type_1
[INFO][2023/04/14 09:40:44 AM] Writing labels/obj_type_1
[INFO][2023/04/14 09:40:44 AM] Loading objects/obj_type_1 (25615, 5) (25615 filtered: None)
[INFO][2023/04/14 09:40:44 AM] Writing properties/obj_type_1/area (25615,)
[INFO][2023/04/14 09:40:44 AM] Writing properties/obj_type_1/major_axis_length (25615,)
[INFO][2023/04/14 09:40:44 AM] Writing properties/obj_type_1/minor_axis_length (25615,)
[INFO][2023/04/14 09:40:44 AM] Writing properties/obj_type_1/orientation (25615,)
[INFO][2023/04/14 09:40:44 AM] Writing properties/obj_type_1/mean_intensity-0 (25615,)
[INFO][2023/04/14 09:40:44 AM] Writing properties/obj_type_1/mean_intensity-1 (25615,)
[INFO][2023/04/14 09:40:44 AM] Writing tracks/obj_type_1
[INFO][2023/04/14 09:40:44 AM] Writing dummies/obj_ty

In [149]:
row, column

(5, 9)

continue with rest of iterations

In [180]:
import os

In [ ]:
### iterate over positions
for (row, column), info in tqdm(assay_layout.iterrows(), 
                                desc = 'Progress through positions',
                                total = len(assay_layout)):
    
    # if tracking file exists, skip over position
    if os.path.exists(f'/mnt/DATA/macrohet/labels/full_localisation/upscale_{row, column}.h5'):
        continue
    else:
        print(f'Processing position {row, column}')
    
    # tile images
    images = tile.compile_mosaic(image_dir, 
                                 metadata, 
                                 row, 
                                 column, 
                                 set_plane = 'sum_proj',
                                 ).astype(np.uint16)

    # segment images from gfp channel only, dask to match intensity image
    masks = da.stack([segment(frame) 
                      for frame in tqdm(images[:,0,...], 
                                        desc = 'Segmenting')])

    # reshape intensity image to be gfp, rfp on last axis for regionprops
    intensity_image = np.stack([images[:,0,...], 
                                images[:,1,...]], axis = -1)

    # localise objects
    objects = localise(masks, 
                       intensity_image, 
                       )

    # filter out small objects
    objects = [o for o in objects if o.properties['area'] > 500]

    # track on upscaled config fn
    tracks = track(objects, masks, config_fn, search_radius = 100)

    # save out 
    with btrack.dataio.HDF5FileHandler(f'/mnt/DATA/macrohet/labels/full_localisation/upscale_{row, column}.h5', 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
            writer.write_objects(objects)
            writer.write_tracks(tracks)
            writer.write_segmentation(masks)


Progress through positions:   0%|          | 0/24 [00:00<?, ?it/s]

Processing position (6, 4)


Segmenting:   0%|          | 0/75 [00:00<?, ?it/s]

[INFO][2023/04/14 12:34:26 PM] Localizing objects from segmentation...
[INFO][2023/04/14 12:34:26 PM] Found intensity_image data
[INFO][2023/04/14 12:49:34 PM] Objects are of type: <class 'dict'>
[INFO][2023/04/14 12:49:34 PM] ...Found 44860 objects in 75 frames.
[INFO][2023/04/14 12:49:35 PM] Loaded btrack: /home/dayn/analysis/BayesianTracker/btrack/libs/libtracker.so
[INFO][2023/04/14 12:49:35 PM] btrack (v0.5.0) library imported
[INFO][2023/04/14 12:49:35 PM] Starting BayesianTracker session
[INFO][2023/04/14 12:49:35 PM] Loading configuration file: /home/dayn/analysis/BayesianTracker/models/cell_config.json
[INFO][2023/04/14 12:49:35 PM] Objects are of type: <class 'list'>
[INFO][2023/04/14 12:49:35 PM] Starting tracking... 
[INFO][2023/04/14 12:49:35 PM] Update using: ['VISUAL', 'MOTION']
[INFO][2023/04/14 12:49:36 PM] Tracking objects in frames 0 to 24 (of 75)...
[INFO][2023/04/14 12:49:43 PM]  - Timing (Bayesian updates: 66.58ms, Linking: 3.31ms)
[INFO][2023/04/14 12:49:43 PM]  

GLPK Integer Optimizer 5.0
25356 rows, 20082 columns, 27508 non-zeros
20082 integer variables, all of which are binary
Preprocessing...
12678 rows, 20082 columns, 27508 non-zeros
20082 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 12678
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
12678 rows, 20082 columns, 27508 non-zeros
*     0: obj =   7.053033900e+04 inf =   0.000e+00 (4044)
Perturbing LP to avoid stalling [1758]...
Removing LP perturbation [4076]...
*  4076: obj =   4.593641013e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  4076: mip =     not found yet >=              -inf        (1; 0)
+  4076: >>>>>   4.593641013e+04 >=   4.593641013e+04   0.0% (1; 0)
+  4076: mip =   4.593641013e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPT

[INFO][2023/04/14 12:49:54 PM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/upscale_(6, 4).h5...
[INFO][2023/04/14 12:49:54 PM] Writing objects/obj_type_1
[INFO][2023/04/14 12:49:54 PM] Writing labels/obj_type_1
[INFO][2023/04/14 12:49:54 PM] Loading objects/obj_type_1 (29778, 5) (29778 filtered: None)
[INFO][2023/04/14 12:49:55 PM] Writing properties/obj_type_1/area (29778,)
[INFO][2023/04/14 12:49:55 PM] Writing properties/obj_type_1/major_axis_length (29778,)
[INFO][2023/04/14 12:49:55 PM] Writing properties/obj_type_1/minor_axis_length (29778,)
[INFO][2023/04/14 12:49:55 PM] Writing properties/obj_type_1/orientation (29778,)
[INFO][2023/04/14 12:49:55 PM] Writing properties/obj_type_1/mean_intensity-0 (29778,)
[INFO][2023/04/14 12:49:55 PM] Writing properties/obj_type_1/mean_intensity-1 (29778,)
[INFO][2023/04/14 12:49:55 PM] Writing tracks/obj_type_1
[INFO][2023/04/14 12:49:55 PM] Writing dummies/obj_type_1
[INFO][2023/04/14 12:49:55 PM] Writing LBEP/obj_type_1
[I

Processing position (6, 5)


Segmenting:   0%|          | 0/75 [00:00<?, ?it/s]

In [182]:
btrack.__version__

'0.5.0'

# Trying to fix IO bug from other notebook

In [188]:
for (row, column), info in tqdm(assay_layout.iterrows(), total = len(assay_layout)):
    # load tracks and objects from original file
    with btrack.dataio.HDF5FileHandler(f"/mnt/DATA/macrohet/labels/full_localisation/downscaled/({row},{column}).h5", "r", obj_type="obj_type_1") as hdf:
            downscale_tracks = hdf.tracks
            downscale_obj = hdf.objects
    # load cleaned masks
    with btrack.dataio.HDF5FileHandler(f'/mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/{row,column}.h5', "r", obj_type="obj_type_1") as hdf:
            downscale_seg = hdf.segmentation
            
    # try writing out 
    with btrack.dataio.HDF5FileHandler(f'/mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned_fix/{row,column}.h5', 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:

            writer.write_objects(downscale_obj)
            writer.write_tracks(downscale_tracks)
            writer.write_segmentation(downscale_seg)

  0%|          | 0/24 [00:00<?, ?it/s]

[INFO][2023/04/17 09:50:21 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(3,4).h5...
[INFO][2023/04/17 09:50:21 AM] Loading tracks/obj_type_1
[INFO][2023/04/17 09:50:21 AM] Loading LBEP/obj_type_1
[INFO][2023/04/17 09:50:21 AM] Loading objects/obj_type_1 (25554, 5) (25554 filtered: None)
[INFO][2023/04/17 09:50:22 AM] Loading objects/obj_type_1 (25554, 5) (25554 filtered: None)
[INFO][2023/04/17 09:50:22 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(3,4).h5
[INFO][2023/04/17 09:50:22 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(3, 4).h5...
[INFO][2023/04/17 09:50:23 AM] Loading segmentation (75, 1200, 1200)
[INFO][2023/04/17 09:50:23 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(3, 4).h5
[INFO][2023/04/17 09:50:23 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned_fix/(3, 4).h5...
[INFO][2023/04/17 09:50:23 AM]

[INFO][2023/04/17 09:50:42 AM] Loading tracks/obj_type_1
[INFO][2023/04/17 09:50:42 AM] Loading LBEP/obj_type_1
[INFO][2023/04/17 09:50:42 AM] Loading objects/obj_type_1 (29074, 5) (29074 filtered: None)
[INFO][2023/04/17 09:50:42 AM] Loading objects/obj_type_1 (29074, 5) (29074 filtered: None)
[INFO][2023/04/17 09:50:42 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(3,8).h5
[INFO][2023/04/17 09:50:42 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(3, 8).h5...
[INFO][2023/04/17 09:50:43 AM] Loading segmentation (75, 1200, 1200)
[INFO][2023/04/17 09:50:43 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(3, 8).h5
[INFO][2023/04/17 09:50:43 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned_fix/(3, 8).h5...
[INFO][2023/04/17 09:50:43 AM] Writing objects/obj_type_1
[INFO][2023/04/17 09:50:43 AM] Writing labels/obj_type_1
[INFO][2023/04/17 09:50:43 AM] 

[INFO][2023/04/17 09:50:58 AM] Loading LBEP/obj_type_1
[INFO][2023/04/17 09:50:58 AM] Loading objects/obj_type_1 (29619, 5) (29619 filtered: None)
[INFO][2023/04/17 09:51:00 AM] Loading objects/obj_type_1 (29619, 5) (29619 filtered: None)
[INFO][2023/04/17 09:51:00 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(4,6).h5
[INFO][2023/04/17 09:51:00 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(4, 6).h5...
[INFO][2023/04/17 09:51:00 AM] Loading segmentation (75, 1200, 1200)
[INFO][2023/04/17 09:51:00 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(4, 6).h5
[INFO][2023/04/17 09:51:00 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned_fix/(4, 6).h5...
[INFO][2023/04/17 09:51:01 AM] Writing objects/obj_type_1
[INFO][2023/04/17 09:51:01 AM] Writing labels/obj_type_1
[INFO][2023/04/17 09:51:01 AM] Loading objects/obj_type_1 (29619, 5) (29619 filtered: No

[INFO][2023/04/17 09:51:19 AM] Loading objects/obj_type_1 (22633, 5) (22633 filtered: None)
[INFO][2023/04/17 09:51:19 AM] Loading objects/obj_type_1 (22633, 5) (22633 filtered: None)
[INFO][2023/04/17 09:51:19 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(5,4).h5
[INFO][2023/04/17 09:51:19 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(5, 4).h5...
[INFO][2023/04/17 09:51:20 AM] Loading segmentation (75, 1200, 1200)
[INFO][2023/04/17 09:51:20 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(5, 4).h5
[INFO][2023/04/17 09:51:20 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned_fix/(5, 4).h5...
[INFO][2023/04/17 09:51:20 AM] Writing objects/obj_type_1
[INFO][2023/04/17 09:51:20 AM] Writing labels/obj_type_1
[INFO][2023/04/17 09:51:20 AM] Loading objects/obj_type_1 (22633, 5) (22633 filtered: None)
[INFO][2023/04/17 09:51:21 AM] Writing properties/o

[INFO][2023/04/17 09:51:38 AM] Loading objects/obj_type_1 (26077, 5) (26077 filtered: None)
[INFO][2023/04/17 09:51:39 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(5,8).h5
[INFO][2023/04/17 09:51:39 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(5, 8).h5...
[INFO][2023/04/17 09:51:39 AM] Loading segmentation (75, 1200, 1200)
[INFO][2023/04/17 09:51:39 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(5, 8).h5
[INFO][2023/04/17 09:51:39 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned_fix/(5, 8).h5...
[INFO][2023/04/17 09:51:40 AM] Writing objects/obj_type_1
[INFO][2023/04/17 09:51:40 AM] Writing labels/obj_type_1
[INFO][2023/04/17 09:51:40 AM] Loading objects/obj_type_1 (26077, 5) (26077 filtered: None)
[INFO][2023/04/17 09:51:40 AM] Writing properties/obj_type_1/area (26077,)
[INFO][2023/04/17 09:51:40 AM] Writing properties/obj_type_1/major_a

[INFO][2023/04/17 09:51:56 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/(6,6).h5
[INFO][2023/04/17 09:51:56 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(6, 6).h5...
[INFO][2023/04/17 09:51:57 AM] Loading segmentation (75, 1200, 1200)
[INFO][2023/04/17 09:51:57 AM] Closing HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned/(6, 6).h5
[INFO][2023/04/17 09:51:57 AM] Opening HDF file: /mnt/DATA/macrohet/labels/full_localisation/downscaled/cleaned_fix/(6, 6).h5...
[INFO][2023/04/17 09:51:57 AM] Writing objects/obj_type_1
[INFO][2023/04/17 09:51:57 AM] Writing labels/obj_type_1
[INFO][2023/04/17 09:51:57 AM] Loading objects/obj_type_1 (25123, 5) (25123 filtered: None)
[INFO][2023/04/17 09:51:57 AM] Writing properties/obj_type_1/area (25123,)
[INFO][2023/04/17 09:51:57 AM] Writing properties/obj_type_1/major_axis_length (25123,)
[INFO][2023/04/17 09:51:57 AM] Writing properties/obj_type_1/mean_intens

In [184]:
# try writing out 
with btrack.dataio.HDF5FileHandler(f'test.h5', 
                                   'w', 
                                   obj_type='obj_type_1'
                                   ) as writer:

        writer.write_objects(downscale_obj)
        writer.write_tracks(downscale_tracks)
        writer.write_segmentation(downscale_seg)

[INFO][2023/04/17 09:45:26 AM] Opening HDF file: test.h5...
[INFO][2023/04/17 09:45:27 AM] Writing objects/obj_type_1
[INFO][2023/04/17 09:45:27 AM] Writing labels/obj_type_1
[INFO][2023/04/17 09:45:27 AM] Loading objects/obj_type_1 (25237, 5) (25237 filtered: None)
[INFO][2023/04/17 09:45:27 AM] Writing properties/obj_type_1/area (25237,)
[INFO][2023/04/17 09:45:27 AM] Writing properties/obj_type_1/major_axis_length (25237,)
[INFO][2023/04/17 09:45:27 AM] Writing properties/obj_type_1/mean_intensity-0 (25237,)
[INFO][2023/04/17 09:45:27 AM] Writing properties/obj_type_1/mean_intensity-1 (25237,)
[INFO][2023/04/17 09:45:27 AM] Writing properties/obj_type_1/minor_axis_length (25237,)
[INFO][2023/04/17 09:45:27 AM] Writing properties/obj_type_1/orientation (25237,)
[INFO][2023/04/17 09:45:27 AM] Writing tracks/obj_type_1
[INFO][2023/04/17 09:45:27 AM] Writing dummies/obj_type_1
[INFO][2023/04/17 09:45:27 AM] Writing LBEP/obj_type_1
[INFO][2023/04/17 09:45:27 AM] Writing fates/obj_type_1


In [187]:
# try reloading (this is where the original bug is)
with btrack.dataio.HDF5FileHandler(f'test.h5', "r", obj_type="obj_type_1") as hdf:
#         downscale_seg = hdf.segmentation
        downscale_tracks = hdf.tracks
#         downscale_obj = hdf.objects

[INFO][2023/04/17 09:47:52 AM] Opening HDF file: test.h5...
[INFO][2023/04/17 09:47:52 AM] Loading tracks/obj_type_1
[INFO][2023/04/17 09:47:52 AM] Loading LBEP/obj_type_1
[INFO][2023/04/17 09:47:52 AM] Loading objects/obj_type_1 (25237, 5) (25237 filtered: None)
[INFO][2023/04/17 09:47:53 AM] Closing HDF file: test.h5
